In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.multiclass import OneVsOneClassifier
from sklearn.metrics import accuracy_score

In [4]:
file_path = r"C:\Users\dipak\OneDrive\desktop\github_ai\ml_with_python\three_module\Multi-class-Classification\Obesity_level_prediction_dataset.csv"

In [5]:
df = pd.read_csv(file_path)


In [7]:
df.sample(5)

,Gender,Age,Height,Weight,family_history_with_overweight,FAVC,FCVC,NCP,CAEC,SMOKE,CH2O,SCC,FAF,TUE,CALC,MTRANS,NObeyesdad
494,Female,18.000000,1.670000,66.000000,no,yes,3.000000,3.000000,Sometimes,no,2.000000,no,0.000000,0.000000,Sometimes,Public_Transportation,Normal_Weight
1469,Male,18.000000,1.811738,108.897324,yes,yes,2.000000,1.202179,Sometimes,no,2.362930,no,1.000000,1.475740,no,Public_Transportation,Obesity_Type_I
770,Male,21.029633,1.607082,67.722222,yes,yes,2.000000,3.691226,no,no,3.000000,no,1.228136,0.335200,Sometimes,Public_Transportation,Overweight_Level_I
1526,Male,31.457413,1.874070,128.867444,yes,yes,2.956297,3.000000,Sometimes,yes,1.275100,no,0.901924,1.875023,Sometimes,Automobile,Obesity_Type_II
1072,Male,19.443639,1.744733,87.279890,yes,yes,2.442536,3.000000,Sometimes,no,2.825629,no,3.000000,0.000000,Sometimes,Public_Transportation,Overweight_Level_II


In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2111 entries, 0 to 2110
Data columns (total 17 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   Gender                          2111 non-null   object 
 1   Age                             2111 non-null   float64
 2   Height                          2111 non-null   float64
 3   Weight                          2111 non-null   float64
 4   family_history_with_overweight  2111 non-null   object 
 5   FAVC                            2111 non-null   object 
 6   FCVC                            2111 non-null   float64
 7   NCP                             2111 non-null   float64
 8   CAEC                            2111 non-null   object 
 9   SMOKE                           2111 non-null   object 
 10  CH2O                            2111 non-null   float64
 11  SCC                             2111 non-null   object 
 12  FAF                             21

In [9]:
df.describe()

,Age,Height,Weight,FCVC,NCP,CH2O,FAF,TUE
count,2111.000000,2111.000000,2111.000000,2111.000000,2111.000000,2111.000000,2111.000000,2111.000000
mean,24.312600,1.701677,86.586058,2.419043,2.685628,2.008011,1.010298,0.657866
std,6.345968,0.093305,26.191172,0.533927,0.778039,0.612953,0.850592,0.608927
min,14.000000,1.450000,39.000000,1.000000,1.000000,1.000000,0.000000,0.000000
25%,19.947192,1.630000,65.473343,2.000000,2.658738,1.584812,0.124505,0.000000
50%,22.777890,1.700499,83.000000,2.385502,3.000000,2.000000,1.000000,0.625350
75%,26.000000,1.768464,107.430682,3.000000,3.000000,2.477420,1.666678,1.000000
max,61.000000,1.980000,173.000000,3.000000,4.000000,3.000000,3.000000,2.000000
